# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


### The event_datafile_new.csv now contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [6]:
from cassandra.cluster import Cluster

try: 
    # This should make a connection to a Cassandra instance your local machine 
    # (127.0.0.1)
    cluster = Cluster(['127.0.0.1'])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [7]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")
except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

# Part II. Time to answer important query questions

### Query 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

This query focuses on session history, so the table will be named accordingly.

Filtering needs to be done by session_id AND item_in_session. 

Looking at the data, it's safe to assume that both aforementioned columns would identify row uniquely, so both columns will be included for primary key. 

In [9]:
session_history_create = """CREATE TABLE IF NOT EXISTS session_history(
        session_id int, 
        item_in_session int, 
        artist_name text,
        song_title text, 
        song_length float, 
        PRIMARY KEY(session_id, item_in_session));"""

try:
    session.execute(session_history_create)
except Exception as e:
    print(e)                    

In [10]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_history(session_id, item_in_session, artist_name, song_title, song_length)"
        query = query + "VALUES(%s, %s, %s, %s, %s);"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

### Let's test query 1 with an example 

In [11]:
query = """SELECT artist_name, song_title, song_length 
                FROM session_history 
                WHERE session_id=338 AND item_in_session=4;"""
try:
    result = pd.DataFrame(list(session.execute(query)))
    print(result.to_string())
except Exception as e:
    print(e)    

  artist_name                       song_title  song_length
0   Faithless  Music Matters (Mark Knight Dub)   495.307312


### Query 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

This query focuses on specific user session history. 

We will be retrieving specific listening details by user_id and session_id. 

To support query, user_id, session_id and item_in_session will be used for primary key (where item_in_session is needed to give sorted output.  

In [12]:
user_session_history_create = """
    CREATE TABLE IF NOT EXISTS user_session_history(
        user_id int,
        session_id int, 
        item_in_session int, 
        artist_name text,
        song_title text, 
        user_first_name text,
        user_last_name text,
        PRIMARY KEY(user_id, session_id, item_in_session));"""

try:
    session.execute(user_session_history_create)
except Exception as e:
    print(e)       

In [13]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_session_history(user_id, session_id, item_in_session, artist_name, song_title, user_first_name, user_last_name)"
        query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s);"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

### Let's test query 2 with an example 

In [14]:
query = """SELECT artist_name, song_title, user_first_name, user_last_name
                FROM user_session_history 
                WHERE user_id=10 AND session_id=182;"""

try:
    result = pd.DataFrame(list(session.execute(query)))
    print(result.to_string())
except Exception as e:
    print(e)    

         artist_name                                         song_title user_first_name user_last_name
0   Down To The Bone                                 Keep On Keepin' On          Sylvie           Cruz
1       Three Drives                                        Greece 2000          Sylvie           Cruz
2  Sebastien Tellier                                          Kilometer          Sylvie           Cruz
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...          Sylvie           Cruz


### Query 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

This query focuses on song history, highlighting which songs are being listened to by which users. 

We will be retrieving specific listening details by song_title. 

To support query, song_title and user_id will be used for primary key. 

Note: Song names might not be unique (e.g. different artists might have same song names). To remove ambiguity, in future it might makes sense to specify more details about this query. 

In [15]:
song_user_history_create = """
    CREATE TABLE IF NOT EXISTS song_user_history(
        user_id int,
        song_title text, 
        user_first_name text,
        user_last_name text,
        PRIMARY KEY(song_title, user_id));"""

try:
    session.execute(song_user_history_create)
except Exception as e:
    print(e)    

In [16]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_user_history(user_id, song_title, user_first_name, user_last_name)"
        query = query + "VALUES(%s, %s, %s, %s);"
        session.execute(query, (int(line[10]), line[9], line[1], line[4]))

### Let's test query 3 with an example 

In [17]:
query = """SELECT user_first_name, user_last_name
                FROM song_user_history 
                WHERE song_title='All Hands Against His Own';"""

try:
    result = pd.DataFrame(list(session.execute(query)))
    print(result.to_string())
except Exception as e:
    print(e)    

  user_first_name user_last_name
0      Jacqueline          Lynch
1           Tegan         Levine
2            Sara        Johnson


### Dropping the tables before closing out the sessions

In [18]:
drop_session_history = "DROP TABLE IF EXISTS session_history;"
session.execute(drop_session_history)
drop_user_session_history = "DROP TABLE IF EXISTS user_session_history;"
session.execute(drop_user_session_history)
drop_song_user_history = "DROP TABLE IF EXISTS song_user_history;"
session.execute(drop_song_user_history)

# Don't forget to close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()